In [158]:
import datetime
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision

from torchvision import datasets, transforms

# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [159]:
############## TENSORBOARD ########################
import sys
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
writer_dir = 'runs/mnist_'+ datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
writer = SummaryWriter(writer_dir)
###################################################

In [160]:
# path to store data and/or load from
!mkdir -p datasets
data_path="./datasets"

In [161]:
train_data = datasets.MNIST(root=data_path, train=True, 
                            transform=transforms.ToTensor(), download=True)
x_train, y_train = train_data.data,train_data.targets
print(f'train data shape: {x_train.shape, y_train.shape}')

val_data = datasets.MNIST(root=data_path, train=False, 
                          transform=transforms.ToTensor(), download=True)
x_val,y_val = val_data.data, val_data.targets
print(f'val data shape: {x_val.shape, y_val.shape}')


train data shape: (torch.Size([60000, 28, 28]), torch.Size([60000]))
val data shape: (torch.Size([10000, 28, 28]), torch.Size([10000]))


In [162]:
# define hyper parameters
input_size    = x_train.shape[1] * x_train.shape[2]
num_classes   = len( train_data.targets.unique() )

hidden_size   = 500
num_epochs    = 1
batch_size    = 64
learning_rate = 0.001

In [163]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=batch_size,
                                         shuffle=False)                                           

In [164]:
examples = iter(train_loader)
samples, labels = examples.next()

In [165]:
samples.shape

torch.Size([64, 1, 28, 28])

In [166]:
img_grid = torchvision.utils.make_grid(samples)

# info for tensorboard
writer.add_image('mnist_images', img_grid)
# writer.close()

In [167]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [168]:
model = NeuralNet(input_size, hidden_size, num_classes)
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# info for tensorboard
writer.add_graph(model, samples.reshape(-1,28*28))
# writer.close()

In [169]:
# training loop
n_total_steps = len(train_loader)

# info for tensorboard
running_loss = running_correct = 0

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        loss    = criterion(outputs, labels)

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred_val, pred_idx = torch.max(outputs, 1)

        running_loss += loss.item()
        running_correct += (pred_idx == labels).sum()

        if (i+1) % 100 == 0:
            print(f'epoch: {epoch+1}|{num_epochs}, step: {i+1}|{n_total_steps}, '
                  f'loss: {loss.item():.4f}')
            writer.add_scalar('training loss', running_loss/100, epoch * n_total_steps + i)
            writer.add_scalar('accuracy ', running_correct/100, epoch * n_total_steps + i)
            running_loss = running_correct = 0

epoch: 1|1, step: 100|938, loss: 0.4063
epoch: 1|1, step: 200|938, loss: 0.4335
epoch: 1|1, step: 300|938, loss: 0.2882
epoch: 1|1, step: 400|938, loss: 0.1359
epoch: 1|1, step: 500|938, loss: 0.4349
epoch: 1|1, step: 600|938, loss: 0.0736
epoch: 1|1, step: 700|938, loss: 0.2898
epoch: 1|1, step: 800|938, loss: 0.1462
epoch: 1|1, step: 900|938, loss: 0.0512


In [170]:
# test iteration
class_labels = []
class_preds = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in val_loader:
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)

        # calculate predictions
        outputs = model(images)
        pred_val, pred_idx = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (pred_idx == labels).sum()

        class_probs_batch = [F.softmax(output, dim=0) for output in outputs]

        class_preds.append(class_probs_batch)
        class_labels.append(pred_idx)

    # 10000, 10, and 10000, 1
    # stack concatenates tensors along a new dimension
    # cat concatenates tensors in the given dimension
    class_preds = torch.cat([torch.stack(batch) for batch in class_preds])
    class_labels = torch.cat(class_labels)        
    print(f'acc: {100 * n_correct/n_samples}')

    ############## TENSORBOARD ########################
    classes = range(10)
    for i in classes:
        labels_i = class_labels == i
        preds_i = class_preds[:, i]
        writer.add_pr_curve(str(i), labels_i, preds_i, global_step=0)
        writer.close()
    ###################################################

acc: 96.2699966430664
